In [12]:
import pandas as pd

# Load data
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

# Merge transaction data with product data
merged_data = pd.merge(transactions_df, products_df, on='ProductID')

# Merge customer data with the above merged data
customer_transactions = pd.merge(merged_data, customers_df, on='CustomerID')

# Check the structure of the merged data
customer_transactions.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Brittany Harvey,Asia,2024-09-04
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Kathryn Stevens,Europe,2024-04-04
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Travis Campbell,South America,2024-04-11
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Timothy Perez,Europe,2022-03-15


In [14]:
# Total value spent by each customer
customer_summary = customer_transactions.groupby('CustomerID').agg(
    total_spent=('TotalValue', 'sum'),
    frequency=('TransactionID', 'count')
).reset_index()

# Merge back with customer profile information
customer_summary = pd.merge(customer_summary, customers_df, on='CustomerID')

# Check the feature summary
customer_summary.head()

,CustomerID,total_spent,frequency,CustomerName,Region,SignupDate
0,C0001,3354.52,5,Lawrence Carroll,South America,2022-07-10
1,C0002,1862.74,4,Elizabeth Lutz,Asia,2022-02-13
2,C0003,2725.38,4,Michael Rivera,South America,2024-03-07
3,C0004,5354.88,8,Kathleen Rodriguez,South America,2022-10-09
4,C0005,2034.24,3,Laura Weber,Asia,2022-08-15


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Standardize numerical features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_summary[['total_spent', 'frequency']])

# Compute cosine similarity between all customers
cosine_sim = cosine_similarity(scaled_features)

# Convert the cosine similarity matrix into a DataFrame for easier handling
cosine_sim_df = pd.DataFrame(cosine_sim, index=customer_summary['CustomerID'], columns=customer_summary['CustomerID'])

# Check the similarity matrix for the first few customers
cosine_sim_df.head()

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.953612,0.782426,-0.740746,0.775453,-0.517069,0.619098,-0.366424,0.922045,0.963577,...,0.990353,0.933706,0.625366,-0.410512,-0.713431,-0.766735,0.797223,0.825839,0.943515,-0.980100
C0002,0.953612,1.000000,0.933600,-0.908617,0.929558,-0.235412,0.826788,-0.629527,0.995800,0.999386,...,0.902699,0.998177,0.831266,-0.665972,-0.891281,-0.537913,0.941973,0.957287,0.999490,-0.874878
C0003,0.782426,0.933600,1.000000,-0.997929,0.999938,0.128465,0.973447,-0.866130,0.962485,0.920468,...,0.688587,0.953523,0.975249,-0.889047,-0.994579,-0.200135,0.999709,0.997327,0.944564,-0.643238
C0004,-0.740746,-0.908617,-0.997929,1.000000,-0.998582,-0.191994,-0.986156,0.896489,-0.943037,-0.893421,...,-0.640514,-0.932165,-0.987453,0.916656,0.999208,0.136694,-0.996086,-0.990561,-0.921487,0.592652
C0005,0.775453,0.929558,0.999938,-0.998582,1.000000,0.139485,0.975933,-0.871635,0.959408,0.916065,...,0.680481,0.950114,0.977647,-0.894083,-0.995674,-0.189228,0.999379,0.996453,0.940855,-0.634684


In [18]:
# Create a dictionary to store the lookalikes and similarity scores
lookalike_dict = {}

# For customers C0001 to C0020
for customer_id in customer_summary['CustomerID'][:20]:
    # Get similarity scores for the current customer
    sim_scores = cosine_sim_df[customer_id]
    
    # Sort and get the top 3 most similar customers (excluding the customer itself)
    top_similar = sim_scores.drop(customer_id).sort_values(ascending=False).head(3)
    
    # Store the lookalikes and scores
    lookalike_dict[customer_id] = [(cust_id, score) for cust_id, score in top_similar.items()]

# Convert the dictionary to a DataFrame
lookalike_df = pd.DataFrame.from_dict(lookalike_dict, orient='index')

# Save the lookalike mapping to a CSV file
lookalike_df.to_csv('Mittali_Lookalike.csv', header=False)